<a href="https://colab.research.google.com/github/starelena/prj_food_store/blob/main/prj_food_store_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version 3

The warehouse mantain trace of all the acquiring stock

# Version 2

I develop the suggetions:
- follow the requirement:
  - add qantity to the stocked product, in case of acquiring
  - subtract quantity to the stocked product, in case of selling
- persistence data in a generic path;
- don't load data at the first excution
- check the positive of quantity and prices
- semplifier code: don't use Company, Store or Category classes.

# **Requirements**
## **Software gestionale per negozio di prodotti vegani**


**BioMarket s.a.s** ti assume per sviluppare un piccolo software gestionale per la loro nuova bottega in Via Tan 6. Il software deve avere le seguenti funzionalità:
Registrare nuovi prodotti, con nome, quantità, prezzo di vendita e prezzo di acquisto.
Elencare tutti i prodotti presenti.
Registrare le vendite effettuate.
Mostrare i profitti lordi e netti.
Mostrare un menu di aiuto con tutti i comandi disponibili.

Il software è testuale, quindi utilizzabile da riga di comando.



1.  Il programma deve essere persistente, cioè le informazioni inserite
dall'utente devono essere mantenute tra diverse esecuzioni del
programma, per fare questo puoi utilizzare un file di testo scegliendo tu
che tipo di codifica utilizzare per le informazioni.
2.  Assicurati che gli input inseriti dall'utente siano validi, ad esempio che i numeri siano effettivamente numeri, gestisci i casi non validi con
eccezioni e messagi di errore.
3. Durante un acquisto, verifica che i prodotti acquistati siano
effettivamente presenti nel magazzino, nel caso negativo mostra
all'utente un messaggio di errore
4. Durante l'aggiunta in magazzino, verifica se il prodotto da aggiungere è già presente magazzino, nel caso positivo aggiungi la quantità a quella
già presente in magazzino, in questo caso non serve specificare di
nuovo il prezzo di acquisto e di vendita, altrimenti registralo come un
nuovo prodotto.
5. Il profitto lordo è il totale delle vendite, cioè tutto ciò che i clienti hanno
pagato, il profitto netto invece è pari al profitto lordo meno il costo di
acquisto per i prodotti.


PROBLEM</br>
During the requirements acquiring, I'll probably discuss about point 4.</br>
The 4 requirement is in conflict with point 1 and also introduces a week point in a application.</b>
I think that when I associate a quantity or prices (very fast mutable quantity during time) to a product  in really we focus on a stock action in warehouse. So when I consider a bought stock in the past, I coulden't change any attribute to not modify the history, becouse any stock could have different bought price and different price and different quntity.</br>
So I think It's not complitely correct change the quantity about a past stock action.</br>
I'm sure that for the moment we don't manage discounts. In this case I have to split the stock to change the final price for the remaining quantity. But this not the case. </br>

PROPOSED SOLUTION</br>
I am managing the Acquiring Data as a really stock. So I use a dictionary with a unique identifier of the product as key and a list of products stocks. So in the future I could add other services as a trend of the price and a study about marginality.



# Working hypothesis
## 1. Warehouse

I worked as the store warehouse is never empty also when It open the first time.
So at the moment, if the controller doesn't found the input file to load the data, It loads a basic set of stock.

**Possible future requirements:**

- save a file with the date in the name. so it could used as backup and be loaded it in case of any problems;

- archive the stock of products (InProduct), if the attribute *is_sold_out*  is true. this could be faster to decide wich stock is selling-in;

- use the attribute *expiring_date* to decide a **Sale**. this require to split the old InProduct. A proposal is:
    - add two attributes to InProduct:</br>
     *part_of_stock* that is 1on1 if the final price doesn't change, 1onN (N is the total numeber in wich the stock is splitted)</br>
     *stock_id* as the reference of the original stock(it could be equal *id* if the stock is not cloned)
    - close the original stock: change *part_of_stock* and set as sold-out
    - clone the InProduct as </br>
      *part_of_stock* as 2onN </br>
      with the different final price,</br>
      with *acq_qty* = *acq_qty* - *sold_qty* .

- price polices: clear rules about how has to change the final price of a remaining product stock if the next stock has a different price.


## 2. A Store
I used OO paradigm. It isn't a fast technic but It is very usefull to idealize the model object in the business area. Some objects are more used and the other could help when expands the services.
First of all I define the model class:
- Company
- Store
- InProduct (almost a Stock)
- SoldProduct
- Product for Sale

I introduce the enum Category becouse It could help us to analyze in/out product Data.</br>
I introduce tha data of Acquiring to have an historical acquiring price trend.</br>
I introduce a product expiring date to decide a sale price near this date.</br>

NOTE:</br>
I need to indicate a unique product identifier to have a data structure usefull during the life-cicle of the program.
I used EAN.</br>
**EAN** or European Article Number (International Article Number) is a standard to describe with a bar-code a number system used in the global business to indicate a specific kind of product of a particular producer.
this standard is used also in the system Global Trade Item Number by GS1 organization.
The EAN-13 is a code formed by 13 numeric character, diveded in four parts:
- Prefix GS1 - 3 digit number (300-330 to indicate Italy)
- ProducerCode - variable length
- ProductCode - variable length
- Last Charcter - 1 digit: a check number calculated by ActiveBarcode

https://it.activebarcode.com/codes/ean13

**Possible future requirements:**
- read barcode of the product
- when the store is closed, a batch job warn the owner about near exparing-date products, to decide if organize a Sale or not.
- organize a file of SoldProduct for day
- realize a dashboard of Sold Product day by day and visualize other KPI(this requirement is to go into details) (analyze the history of acquiring price)
- internationalization of output string
- internazionalization of log too

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import logging

LOG_PATH = "./"
LOG_FILE = "com_store.log"

logger = logging.getLogger("APP_STORE")
logger.setLevel(logging.DEBUG)
file_handler = logging.FileHandler(LOG_PATH + LOG_FILE)
format_str = "%(asctime)s [%(levelname)s]: %(message)s in %(pathname)s:%(lineno)d"
file_formatter = logging.Formatter(format_str)
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)
logger.info("Abilitato log in " + LOG_PATH + LOG_FILE)

INFO:APP_STORE:Abilitato log in ./com_store.log


In [ ]:
DATE_FORMAT = "%Y-%m-%d"

PATH              = "./"
FILE_NAME_STOCK   = "products_stock.json"
FILE_NAME_SELL_IN = "products_sell_in.json"

In [ ]:
import random as rd
import uuid
from enum import Enum
from datetime import date
from dataclasses import dataclass, asdict
from __future__ import annotations

class JsonSerializable() :

  def to_json(self) :
    '''
    convert the instance of this class to json and then in a python dict
    '''
    return json.loads( json.dumps(self, indent = 4, default=lambda o: o.__dict__) )


class Company() :
  """
  A group could has different company
  """
  _id = 1
  NAME = "BioMarket s.a.s."
  PIVA = "IT123456789"

class Store() :
  """
  A company could open different Store
  """
  _id = 1
  _company_id = 1
  NAME = "TAN_STORE"
  ADDRESS = "Via Tan 6"

@dataclass
class FoodCategory(Enum) :
  """
  Food Category help us to anylize data
  """
  DRINK = 1
  PROTEIN = 2
  CEREAL = 3
  FRUIT = 4
  VEGETABLE = 5

  @classmethod
  def from_id(cls, id):
    if ( type(id) == "int" ) :
      value = cls(id)
      if isinstance(value, FoodCategory) :
        return value

    return "NOT a Category!"

@dataclass
class InProduct(JsonSerializable) :
  """
  In Product/Food
  """
  def  __init__(self, name, ean, category, bought_qty, acq_price, expiring_date, price, in_date) :
    r_int = rd.randint(10000000000, 99999999999)
    self.id = uuid.uuid4().hex
    self.ean = ean
    if (ean == "0000000000000") :
      self.ean = "330" + str(r_int)
    self.name = name
    self.category = int(category)
    self.bought_qty = int(bought_qty)
    self.acq_price = float(acq_price)

    if isinstance(expiring_date, date) :
      expiring_date = expiring_date.strftime(DATE_FORMAT)
    self.expiring_date =  expiring_date

    self.price = float(price)
    self.is_sold_out = False
    self.sold_qty = 0

    if isinstance(in_date, date) :
      in_date = in_date.strftime(DATE_FORMAT)
    self.added_date = in_date


  def set_added_date(self, s_date) :
    if isinstance(s_date, date) :
      s_date = s_date.strftime(DATE_FORMAT)
    self.added_date =  s_date

  def set_subtracted_date(self, s_date) :
    if isinstance(s_date, date) :
      s_date = s_date.strftime(DATE_FORMAT)
    self.subtracted_date =  s_date

  def get_is_sold_out(self) :
    return self.is_sold_out

  def is_a_unique_sale(self, curr_sold_qty) :
    ret_qty = (self.sold_qty + curr_sold_qty) - self.bought_qty
    logger.debug("remain quantity of a stock: " + str(-ret_qty))
    if ( ret_qty < 0) :
      self.sold_qty = self.sold_qty + curr_sold_qty
      ret_qty = 0
    elif (ret_qty == 0) :
      self.sold_qty = self.sold_qty + curr_sold_qty
      self.is_sold_out = True

    elif (ret_qty > 0) :
      self.sold_qty = self.bought_qty
      self.is_sold_out = True

    return [ret_qty, self.price, self.acq_price]


@dataclass
class SoldProduct(JsonSerializable) :
  """
  Sold Product
  """
  def __init__(self, name, product_ean, sold_qty, sold_price, acq_price, in_date) :
    self.id = uuid.uuid4().hex
    self.name = name
    self.product_ean = product_ean
    self.sold_qty = int(sold_qty)
    self.sold_price = float(sold_price)
    self.acq_price = float(acq_price)
    if ( type(in_date) == date) :
      in_date = in_date.strftime("%Y-%m-%d")
    self.in_date = in_date
    self.tot = 0


#basic TEST
sold_pr = SoldProduct("Latte di Soia", "33073237135065", 15, 2.1, 1.65, "2024-07-25")
sold_pr.tot = 31.5

print(str(sold_pr))



SoldProduct()


In [ ]:
import dataclasses
import json

def in_product_decoder(o):
  try:
    if isinstance(o, dict) and "name" in o :
      new_obj = InProduct(o["name"], o["ean"], o["bought_qty"], o["acq_price"], o["expiring_date"], o["price"], o["added_date"])
      new_obj.added_date = o["added_date"]
      return new_obj
    else:
      return o
  except TypeError as err:
      print ( err )


class ModelEncoder(json.JSONEncoder):

  def default(self, o):
    if dataclasses.is_dataclass(o):
      o_dict = o.to_json()
      return o_dict
    return super().default(o)

class ArrayDecoder(json.JSONDecoder):

  def __init__(self, class_type="InProduct", **kwargs) :
    json.JSONDecoder.__init__(self, **kwargs)
    # define a custom dict parser
    self.parse_array = self.ArrayParser
    # self.parse_dict = self.ObjectParser
    self.parse_costant = self.CostantParser
    # Use the python implementation of the scanner
    self.scan_once = json.scanner.py_make_scanner(self)
    self.class_type = class_type

  def CostantParser(self, s_and_end, scan_once, **kwargs) :
    value, end = json.decoder.JSONArray(s_and_end, scan_once, **kwargs)
    return value, end

  def ArrayParser(self, s_and_end, scan_once, **kwargs) :
    values, end = json.decoder.JSONArray(s_and_end, scan_once, **kwargs)
    logger.debug(values)
    obj_list = []
    for el in values :
      logger.debug(el)
      if ("InProduct" == self.class_type) :
        new_obj = InProduct(el["name"], el["ean"], el["category"], el["bought_qty"], el["acq_price"], el["expiring_date"], el["price"], el["added_date"])
        new_obj.is_sold_out = el["is_sold_out"]
        new_obj.sold_qty = el["sold_qty"]
        obj_list.append(new_obj)
      elif  ("SoldProduct" == self.class_type) :
        new_obj = SoldProduct(el["name"], el["product_ean"], el["sold_qty"], el["sold_price"], el["acq_price"], el["in_date"])
        new_obj.tot = el["tot"]
        obj_list.append(new_obj)

    return obj_list, end

# GUI or Text User Interface

I used a MVC paradigm.</br>
I found usefull to delegate to the controllers all the problems about manage data and delegate to the View class the purpose to interact with the final user.</br>
In particolary, I introduce a Controller (LoadWarehouseController) to load data at the startup of the application to faster the test.


In [ ]:
class NotPositiveError (Exception) :

    def __init__(self, value):
      self.value = value

    # __str__ is to print() the value
    def __str__(self):
      return(repr(self.value))


In [ ]:
class GuiMessage(object):

  # COMMAND Line Service
  INSERT_CMD_MSG  = "\nInserisci un comando: "
  ABILYFIED_CMDS  = "\nI comandi abilitati sono i seguenti: "
  WARN_DIGIT_HELP = "I commandi sono stringhe. digita 'aiuto' per l'elenco comandi"
  WARN_NO_CMD     = "Comando non implementato"
  WARN_NO_ABILYFIED_CMD = "Comando non abilitato"
  # INIT DATA
  INIT_FIRST_DATA = "Primo accesso: alcuni prodotti presenti in magazzino."
  INIT_STOCK   = "Prodotti presenti in magazzino."
  INIT_SELL_IN = "Caricato storico vendite"
  INIT_NO_STOCK   = "PROBLEMA: Prodotti non presenti in magazzino."
  INIT_NO_SELL_IN = "Non presente storico vendite."
  INIT_FILE_NOT_FOUND = "File non presente {0}"
  # LIST Service
  LIST_CURRENT_PRODUCT = "\nI Prodotti presenti sono: \n{0:35}\t{1:10}\tPREZZO".format("PRODOTTO", "QUANTITà")
  WARN_LIST_NO_PRODUCT = "Non sono presenti prodotti in Magazzino."
  # ADD Service
  ADD_CHOISE    = "Seleziona fra i prodotti: {0} o aggiungi uno nuovo:"
  ADD_NAME      = "Nome del prodotto: "
  ADD_QTY       = "Quantità: "
  ADD_ACQ_PRICE = "Prezzo di acquisto: "
  ADD_PRICE     = "Prezzo di vendita: "
  ADD_RESULT    = "AGGIUNTO: "
  WARN_ADD_NAME      = "Nome del prodotto: attenzione è una stringa alfanumerica senza caratteri speciali"
  WARN_ADD_QTY       = "Quantità: attenzione: è un numero intero"
  WARN_ADD_ACQ_PRICE = "Prezzo di acquisto: è un numero con virgola (.)"
  WARN_ADD_PRICE     = "Prezzo di vendita: è un numero con virgola (.)"
  # SELL Service
  SELL_CHOISE    = "Seleziona fra i prodotti: "
  SELL_NAME      = "Nome del prodotto: "
  SELL_QTY       = "Quantità: "
  SELL_OTHER_PRODUCT = "Aggiungere un altro prodotto ?(si/no): "
  SELL_PRICE     = "Prezzo di vendita: "
  SELL_RESULT_0  = "\nVENDITA REGISTRATA"
  SELL_RESULT_1  = "Totale:\t\t\t\t€{0:.2f}"
  WARN_SELL_NAME = "Nome del prodotto: attenzione è una stringa alfanumerica senza caratteri speciali"
  WARN_SELL_QTY  = "Quantità: attenzione: è un numero intero"
  WARN_SELL_EXCED_QTY = "Quantità richiesta eccede la quantità disponibile: "
  WARN_SELL_NO_DATA = "\nNon sono presenti prodotti per la vendita."
  # REVENUES Service
  REV_PROD_TIT   = "\n{0:35}\t{1:10}\t{2:20}\t{3}\t{4:15}\t{5}".format("PRODOTTO", "QUANTITà", "PROFITTO NETTO","FATTURATO","PREZZO MEDIO di ACQ.", "PREZZO MEDIO VEND.")
  REV_PROD_RES   = "{0:35}\t\t{1}\t\t€{2:.2f}\t€{3:.2f}\t€{4:.2f}\t€{5:.2f}"
  REV_TOT        = "\nProfitto netto=€{0:.2f}\t\tFatturato=€{1:.2f}"
  WARN_REV_NO_DATA = "\nNon sono presenti dati di vendita per calcolare profitti."
  # exit
  QST_EXIT = "Vuoi realmente uscire?"
  BYEBYE = "Bye bye"
  # INPUT
  WARN_NO_POSITIVE_INT = "Il valore inserito deve essere un intero positivo."
  WARN_NO_POSITIVE_FLOAT = "Il valore inserito deve essere un numero con virgola(.) positivo."

  PRODUCTS_LIST = "I prodotti in magazzino sono:"


In [ ]:
from copy import Error
import json


class WarehouseController() :
  """
  Controller used to load a warehouse of products.
  To test the resume functionality
  """

  """
  finally :
  if (self.in_products_dict is None or len( list(self.in_products_dict.keys()) ) == 0) :
    self.in_products_dict = self._get_products_dict()
    logging.info(GuiMessage.INIT_FIRST_DATA)
  """

  """
  To Load Acquired Stock of Products Dictionary and Sold Products Dictionary:
  """
  def load_warehouse(self, file_name_stock = FILE_NAME_STOCK, file_name_sell_in = FILE_NAME_SELL_IN, file_path = PATH) :
    logger.debug("WarehouseController.load_warehouse ")
    self.in_products_dict = dict()
    self.out_products_dict = dict()

    try :
      self.in_products_dict = self.deserialize(file_path + file_name_stock, "InProduct")
      logging.info(GuiMessage.INIT_STOCK)
    except (FileNotFoundError , FileExistsError)  as err:
      self.in_products_dict = dict()
      logging.debug(err.strerror())
      logging.warning(GuiMessage.INIT_NO_STOCK)

    try :
      self.out_products_dict = self.deserialize(file_path + file_name_sell_in, "SoldProduct")
      logging.info(GuiMessage.INIT_SELL_IN)
    except FileNotFoundError :
      self.out_products_dict = dict()
      logging.info(GuiMessage.INIT_NO_SELL_IN)

    return self.in_products_dict, self.out_products_dict

  def _load_basic_json(self) :
    self._json_data = '[{"name":"Latte di Soia","ean":"33073237135065","category":"1","bought_qty":"20","acq_price":"1.50","expiring_date":"2024-07-20","price":"2.10","in_date":"2024-07-20"},{"name":"Latte di Mandorle","ean":"33090153560986","category":"1","bought_qty":"30","acq_price":"1.70","expiring_date":"2024-08-20","price":"2.30","in_date":"2024-07-20"},{"name":"Orzo perlato","ean":"33049467073327","category":"3","bought_qty":"20","acq_price":"1.20","expiring_date":"2024-11-25","price":"1.75","in_date":"2024-07-20"},{"name":"Gallette di Riso","ean":"3303393393390","category":"3","bought_qty":"35","acq_price":"1.30","expiring_date":"2024-07-20","price":"1.99","in_date":"2024-07-20"}]'
    x = json.loads(self._json_data, object_hook=lambda d: InProduct(**d))
    logger.debug("_load_json_00")
    logger.debug(x[0].id, x[0].name, x[0].category, x[0].bought_qty, x[0].price)
    return x

  def _get_products_dict(self) :
    logger.debug("WarehouseController._get_products_dict ")
    self.in_products_dict = dict()
    _read_json_list = self._load_basic_json()
    for pr in _read_json_list:
      dict_tmp = { pr.ean : [pr] }
      self.in_products_dict.update(dict_tmp)
    return  self.in_products_dict

  def serialize(self, completed_file_name, the_dict) :
    try :
      with open(completed_file_name, "w") as fp:
        if (the_dict is None) :
          the_dict = dict()
        json.dump(the_dict, fp, sort_keys=False, ensure_ascii=True, indent=4, cls=ModelEncoder)
    except ValueError :
      logger.error(GuiMessage.INIT_NO_STOCK)

  def deserialize (self, completed_file_name, class_type) :
    try :
      with open(completed_file_name) as fp :
        in_dict = json.load(fp, cls=ArrayDecoder, class_type=class_type)
        logger.debug (in_dict)
        return in_dict
    except ValueError:
      logger.error(GuiMessage.INIT_NO_SELL_IN)
    except FileNotFoundError:
      logger.error(GuiMessage.INIT_NO_SELL_IN)


# basic TEST
# logger.setLevel(logging.ERROR)
# w_ctrl = WarehouseController()

# w_ctrl._get_products_dict()

# print(w_ctrl.in_products_dict)

# w_ctrl.serialize( w_ctrl.PATH + w_ctrl.FILE_NAME_STOCK, w_ctrl.in_products_dict )

# loaded_dict = w_ctrl.deserialize(w_ctrl.PATH + w_ctrl.FILE_NAME_STOCK, "InProduct")

# print( loaded_dict )

In [ ]:
from enum import Enum

class Command:

  def __init__(self, id, name, user_descr, validator = False):
    self.id = id
    self.cmd_str = name
    self.user_descr = user_descr
    self.validator = validator


class Commands(Enum):
  HELP = Command( 0, "aiuto", "manuale utente")
  ADD = Command( 1, "aggiungi", "aggiungi un prodotto al magazzino", True)
  LIST = Command( 2, "elenca", "elenca i prodotto in magazzino")
  SELL = Command( 3, "vendita", "registra una vendita effettuata")
  PROFITS = Command( 4, "profitti", "mostra i profitti totali")
  EXIT = Command( 5, "chiudi", "esci dal programma")

  def __str__(self) :
    return self.value.cmd_str + ":\t" + self.value.user_descr

  def get_by_str(input_str) :
    logger.debug("method: get_by_str")
    key = ""
    for name in Commands.__members__ :
      if (Commands[name].value.cmd_str == input_str) :
        key = name
        break

    logger.debug(str(input_str) + " Key: " + str(key) + "**")
    if key != "" :
      return Commands[key]
    else :
      return None

# basic TEST
# print(Commands.PROFITS.value.cmd_str)

# print(Commands.PROFITS.value.user_descr)

# print(Commands.HELP in Commands)

# print(Commands.get_by_str( "vendita" ))
# print(Commands.get_by_str( "elenca"))

In [ ]:
class CustCost() :
  """
  YES or NO response
  """
  YES = ["sì", "si", "s", "yes", "y"]
  NO  = ["no", "n"]

  """
  Keys of a dict to comunicate to Controller
  """
  K_NAME = "name"
  K_EAN  = "ean"
  K_QTY  = "qty"
  K_ACQ_PRICE = "acq_price"
  K_PRICE     = "price"
  K_IS_NEW_B  = "is_new"

  K_QTY_TOT = "qty_tot"
  K_REV_TOT = "rev_tot"
  K_SALES_TOT = "sales_tot"
  K_ACQ_PRICE_MEAN = "acq_price_mean"
  K_SELLIN_PRICE_MEAN = "sellin_price_mean"



In [ ]:
# import pyinputplus as pyip

class GuiView() :
  """
  Manage input e output
  """

  """
  Abilify commands
  """
  ABILIFY_CMD_LIST = [Commands.HELP, Commands.ADD, Commands.LIST, Commands.SELL, Commands.PROFITS, Commands.EXIT]


  def await_a_cmd(self) :
    logger.debug("method: await_a_cmd")
    in_cmd_str = input(GuiMessage.INSERT_CMD_MSG)
    logger.debug(in_cmd_str)
    if not in_cmd_str.isalpha() :
      print(GuiMessage.WARN_DIGIT_HELP)
    in_cmd_str = in_cmd_str.lower()
    return Commands.get_by_str(in_cmd_str)

  def view_welcome(self) :
    print("\nBENVENUT* da {0!r} nel negozio {1} di {2} ".format( Company.NAME, Store.NAME, Store.ADDRESS) )

  def view_help(self) :
    logger.debug("method: view_help")
    print(GuiMessage.ABILYFIED_CMDS)
    for en in GuiView.ABILIFY_CMD_LIST :
      print(f"- {en.value.cmd_str}:  {en.value.user_descr}")

  def view_exit(self) :
    is_time_to_exit = False
    answer_str = str(input(GuiMessage.QST_EXIT))
    answer_str = answer_str.lower()
    logger.debug(answer_str)
    if (answer_str in CustCost.YES) :
      is_time_to_exit = True
    elif (answer_str in CustCost.NO) :
      is_time_to_exit = False
    logger.debug("is_time_to_exit: " + str(is_time_to_exit))
    return is_time_to_exit

  def view_products_list(self, current_products_list) :
    logger.debug(current_products_list)
    if (len(current_products_list) == 0) :
      print(GuiMessage.WARN_LIST_NO_PRODUCT)
      return

    print(GuiMessage.LIST_CURRENT_PRODUCT)
    for item in current_products_list :
      print(f"{item[0]:35}\t{item[1]:10}\t{item[2]:.2f}")

  def view_add_product(self, products_name_dict) :
    """
    Return a generic dictionary
    """
    ret_obj = dict()
    ret_obj = self._view_input_name(products_name_dict)

    in_qty_int = self._view_input_int(GuiMessage.ADD_QTY, GuiMessage.WARN_ADD_QTY)
    ret_obj[CustCost.K_QTY] = in_qty_int

    in_acq_price_fl = self._view_input_float(GuiMessage.ADD_ACQ_PRICE, GuiMessage.WARN_ADD_ACQ_PRICE)
    ret_obj[CustCost.K_ACQ_PRICE] = in_acq_price_fl

    in_price_fl = self._view_input_float(GuiMessage.ADD_PRICE, GuiMessage.WARN_ADD_PRICE)
    ret_obj[CustCost.K_PRICE] = in_price_fl

    return ret_obj


  def _view_input_name(self, products_dict) :
    products_name_list = list(products_dict.keys())
    if (products_dict is not None and len( products_dict.keys()) > 0) :
      products_name_list = list(products_dict.keys())

    try:
      names_str = self._view_name_list_as_str(products_name_list)
      logger.debug(names_str)
      print("\n" + GuiMessage.ADD_CHOISE.format(names_str))
      in_name_str = str(input(GuiMessage.ADD_NAME))
      logger.debug(in_name_str + "**" + str(in_name_str.replace("-","").replace(" ","").isalnum()) )

      if ( in_name_str.replace("-","").replace(" ","").isalnum()) :
        ret = self.is_input_by_list(in_name_str, products_name_list)
        logger.debug(ret)
        ean_str = "0000000000000"
        is_new = True
        if (ret) :
          in_name_str = ret
          logger.debug("ret: " + str(in_name_str) )
          ean_str = products_dict[in_name_str]
          is_new = False
        logger.debug(in_name_str)
        logger.debug(ean_str)
        return { CustCost.K_NAME: in_name_str, CustCost.K_EAN : ean_str, CustCost.K_IS_NEW_B: is_new}
      else :
        print(GuiMessage.WARN_ADD_NAME)
        return self._view_input_name(products_dict)
    except ValueError:
        print(GuiMessage.WARN_ADD_NAME)
        return self._view_input_name(products_name_dict)


  def _view_input_int(self, in_msg, warn_msg) :
    try :
      in_int = int(input(in_msg))
      if in_int <= 0 :
        raise NotPositiveError(GuiMessage.WARN_NO_POSITIVE_INT)
      return in_int

    except ValueError :
      print(warn_msg)
      return self._view_input_int(in_msg, warn_msg)
    except NotPositiveError as np_err:
      print(np_err)
      return self._view_input_int(in_msg, warn_msg)


  def _view_input_float(self, in_msg, warn_msg) :
    try :
      in_float = float(input(in_msg))
      if in_float <= 0 :
        raise NotPositiveError(GuiMessage.WARN_NO_POSITIVE_FLOAT)
      return in_float

    except ValueError :
      print(warn_msg)
      return self._view_input_float(in_msg, warn_msg)
    except NotPositiveError as np_err:
      print(np_err)
      return self._view_input_float(in_msg, warn_msg)

  def _view_name_list_as_str(self, names_list) :
    n_str = ""
    for idx in range(len(names_list)) :
      if idx != 0 :
        n_str = n_str + ", "
      n_str = n_str + names_list[idx]
    return n_str

  def is_input_by_list(self, in_str, str_list) :
    is_by_list = False
    for st in str_list :
      if st.lower() == in_str.lower() :
        is_by_list = True
        ret_str = st
        return ret_str

    return is_by_list

  def view_add_result(self, added_inprod) :
    print(GuiMessage.ADD_RESULT + added_inprod[CustCost.K_NAME] + " X " + str(added_inprod[CustCost.K_QTY]))

  """
  To SELL products
  products_dict is a dictionary with key == name of products
  and value a dictionary { name: v_n, ean: v_e, qty: v_q, price: v_p}
  """
  def view_sell_product(self, products_dict) :
    """
    Return a generic dictionary
    """
    is_sell_in = True
    ret_obj_list = []

    while (is_sell_in) :
      ret_obj = dict()
      product = self._view_input_name_to_sell(products_dict)
      logger.debug(product)
      name = product[CustCost.K_NAME]
      ret_obj = {CustCost.K_NAME: name, CustCost.K_EAN : product[CustCost.K_EAN]}

      wh_tot_qty = product[CustCost.K_QTY]
      in_qty_int = wh_tot_qty + 10
      while (in_qty_int > wh_tot_qty) :
        in_qty_int = self._view_input_int(GuiMessage.SELL_QTY, GuiMessage.WARN_SELL_QTY)
        if (in_qty_int > wh_tot_qty) :
          logger.warning(GuiMessage.WARN_SELL_EXCED_QTY + str(wh_tot_qty) + " " + name)
          print(GuiMessage.WARN_SELL_EXCED_QTY + str(wh_tot_qty) )

      ret_obj[CustCost.K_QTY] = in_qty_int
      ret_obj_list.append(ret_obj)
      logger.debug(ret_obj_list)
      is_sell_in = self.view_to_continue_sell_in()


    return ret_obj_list

  def view_to_continue_sell_in(self) :
    is_time_to_continue = False
    answer_str = str(input(GuiMessage.SELL_OTHER_PRODUCT))
    answer_str = answer_str.lower()
    logger.debug(answer_str)
    if (answer_str in CustCost.YES) :
      is_time_to_continue = True
    elif (answer_str in CustCost.NO) :
      is_time_to_continue = False
    logger.debug("is_time_to_continue: " + str(is_time_to_continue))
    return is_time_to_continue

  def _view_input_name_to_sell(self, products_dict) :
    products_name_list = list()
    if (products_dict is not None and len( products_dict.keys()) > 0) :
      products_name_list = list(products_dict.keys())

    try:
      names_str = self._view_name_list_as_str(products_name_list)
      logger.debug(names_str)
      print("\n" + GuiMessage.SELL_CHOISE + names_str)
      in_name_str = str(input(GuiMessage.SELL_NAME))

      ret = self.is_input_by_list(in_name_str, products_name_list)
      logger.debug(ret)
      if (ret) :
        in_name_str = ret
        logger.debug("ret: " + str(in_name_str) )
        logger.debug(products_dict)
        product = products_dict[in_name_str]
        logger.debug(product)
        return product
      else :
        print(GuiMessage.WARN_SELL_NAME)
        product = self._view_input_name_to_sell(products_dict)
        return product
    except ValueError:
        print(GuiMessage.WARN_SELL_NAME)
        product = self.view_input_name(products_dict)
        return product

  def view_sold_products_result(self, sold_products_list) :
    print(GuiMessage.SELL_RESULT_0)

    """
    tot_prod: this is the property of a single receipt (TO MODEL)
    """
    tot_sell_in = 0
    for pr in sold_products_list :
      tot_sell_in      += pr.tot
      print("- {0} X {1}\t€{2:.2f}\t\t€{3:.2f}".format( pr.sold_qty, pr.name, pr.sold_price, pr.tot ) )

    print(GuiMessage.SELL_RESULT_1.format(tot_sell_in) )

  def view_revenues(self, revenues_l, revenues) :
    if (revenues_l is not None) :
      print(GuiMessage.REV_PROD_TIT)
      for res in revenues_l :
        print(GuiMessage.REV_PROD_RES.format(res[CustCost.K_NAME], res[CustCost.K_QTY_TOT], res[CustCost.K_REV_TOT], res[CustCost.K_SALES_TOT], res[CustCost.K_ACQ_PRICE_MEAN], res[CustCost.K_SELLIN_PRICE_MEAN]))
    print (GuiMessage.REV_TOT.format(revenues[CustCost.K_REV_TOT], revenues[CustCost.K_SALES_TOT]))

  def view_byebye(self) :
    print(GuiMessage.BYEBYE)

  def view_error(self, error) :
    print(error)

# TO DEBUG
# a_dict = {'Latte di Soia': '33073237135065', 'Latte di Mandorle': '33090153560986', 'Orzo perlato': '33049467073327'}

# gui = GuiView()

# gui.view_add_product(a_dict)

In [ ]:
from datetime import date
import sys
import logging

class GUIController() :

  def __init__(self):
    logger.debug( "method __init__")

    """
    Prepare View: abilify commands
    """
    logger.debug("Init GuiVIew")
    self.gui = GuiView()

    """
    Init WarehouseController and load Data.
    Prepare InProduct structure data as a dictionary with EAN as a key
    and a list of acquiring InProduct
    Prepare SoldProduct structure data as a dictionary with Ean as a key
    and a list of SoldProduct
    I could load Warehouse without parameters.
    If use parameters, the arguments must be filepaths that contain a particulary structure of Json.
    """
    logger.debug("Init WarehouseController")
    self.wh_ctrl = WarehouseController()

    logger.debug("Init Warehouse")
    self.in_products_dict , self.out_products_dict = self.wh_ctrl.load_warehouse()
    logger.debug(self.in_products_dict)
    logger.debug(self.out_products_dict)

    self.gui.view_welcome()

    """
    to have abilified commands list
    to inter-act with a user
    """
    logger.debug("Call manage_input_cmd in default mode (HELP command)")
    self.manage_input_cmd()


  def manage_input_cmd(self, in_cmd_enum=Commands.HELP) :
    logger.debug("enter swich case: " + str(in_cmd_enum))
    try:
      current_cmd = in_cmd_enum
    except(ValueError) :
      logger.warning(GuiMessage.WARN_NO_CMD)

    logger.debug("Current cmd: " + str(current_cmd))

    if not current_cmd in self.gui.ABILIFY_CMD_LIST :
      logger.warning(GuiMessage.WARN_NO_ABILYFIED_CMD)

    match  current_cmd:
      case Commands.HELP:
        logger.info(Commands.HELP)
        self.gui.view_help()

      case Commands.ADD:
        logger.info(Commands.ADD)
        products_name_dict = self._get_products_name_dict()
        add_obj_dict = self.gui.view_add_product(products_name_dict)
        added_obj = self._add_inprod_to_dict(add_obj_dict)
        self.gui.view_add_result(added_obj)

      case Commands.LIST:
        logger.info(Commands.LIST)
        prods_list = self._get_products_to_list()
        self.gui.view_products_list( prods_list )

      case Commands.SELL:
        logger.info(Commands.SELL)
        try :
          products_dict = self._get_products_to_sell()
          sold_objs_list = self.gui.view_sell_product(products_dict)

          sold_products_list = self._add_sold_prods_to_dict(sold_objs_list)
          self.gui.view_sold_products_result(sold_products_list)
        except ValueError as v_err :
          self.gui.view_error(v_err)

      case Commands.PROFITS:
        logger.info(Commands.PROFITS)
        try :
          revenues_l, revenues = self._calc_revenues()
          self.gui.view_revenues(revenues_l, revenues)
        except ValueError as v_err :
          self.gui.view_error(v_err)

      case Commands.EXIT:
        logger.info(Commands.EXIT)
        is_time_to_exit = self.gui.view_exit()
        if (is_time_to_exit) :
          self.wh_ctrl.serialize( PATH + FILE_NAME_STOCK, self.in_products_dict )
          self.wh_ctrl.serialize( PATH + FILE_NAME_SELL_IN, self.out_products_dict )
          self.gui.view_byebye()
          sys.exit()

    """
    To inter-act
    """
    logger.debug("START inter-action with user")
    in_cmd = self.gui.await_a_cmd()
    self.manage_input_cmd(in_cmd)


  """
  To LIST: Prepare data as a list 2dim, to have a total quantity
  """
  def _get_products_to_list(self) :
    current_products = []
    if ( self.in_products_dict is None or len(self.in_products_dict.keys()) == 0 ) :
      return current_products

    for key in self.in_products_dict :
      qty_tot = 0

      for el in self.in_products_dict[key] :
        qty_tot = qty_tot + el.bought_qty - el.sold_qty
        last_price = el.price
        name = el.name

      current_products.append( [name, qty_tot, last_price] )

    return current_products

  """
  To LIST the products with total quantity
  To Know if there is the product I want to sell
  """
  def calc_product_qty(self, key) :
    qty_tot = 0
    for el in self.in_products_dict[key] :
      qty_tot += el.bought_qty

    try :
      for el2 in self.out_products_dict[key] :
        qty_tot -= el2.sold_qty
    except (KeyError) :
      return qty_tot

    return qty_tot

  """
  To ADD a product: a data structure to pass to view_method,
  to suggest the name list and to use ean of a just present product.
  If there is no data in warehouse, it is not a error
  """
  def _get_products_name_dict(self) :
    products_name_dict = dict()

    if ( self.in_products_dict is None or len( self.in_products_dict.keys()) == 0) :
      return products_name_dict

    for kk in self.in_products_dict :
      logger.debug (products_name_dict)
      logger.debug (self.in_products_dict[kk][0].name)
      products_name_dict[self.in_products_dict[kk][0].name] = kk

    return products_name_dict

  """
  To SELL a product: a data structure to pass to view_method,
  to suggest a product and to know if quantity is enought.
  products_dict is a dictionary with key == name of products
  and value a dictionary { name: v_n, ean: v_e, qty: v_q, price: v_p}
  """
  def _get_products_to_sell(self) :

    if ( self.in_products_dict is None or len( self.in_products_dict.keys()) == 0) :
      raise ValueError(GuiMessage.WARN_SELL_NO_DATA)

    products_dict = dict()
    for kk in self.in_products_dict :
      logger.debug (products_dict)
      name = self.in_products_dict[kk][0].name
      logger.debug (name)
      qty = self.calc_product_qty( kk)
      price = self.in_products_dict[kk][-1].price
      products_dict[name] = { CustCost.K_NAME: name, CustCost.K_EAN: kk, CustCost.K_QTY: qty, CustCost.K_PRICE: price}

    return products_dict

  """
  To update the in_products
  """
  def _add_inprod_to_dict(self, input_obj) :
    logger.debug(input_obj)

    if ( self.in_products_dict is None) :
      self.in_products_dict = dict()

    ean = input_obj[CustCost.K_EAN]
    new_inprod = InProduct(input_obj[CustCost.K_NAME], ean, 1, input_obj[CustCost.K_QTY], input_obj[CustCost.K_ACQ_PRICE], date.today(), input_obj[CustCost.K_PRICE], date.today() )

    if ean == "0000000000000" :
      ean = new_inprod.ean
      self.in_products_dict[ean] = [new_inprod]
    else :
      the_list = self.in_products_dict[ean]
      the_list = the_list.append(new_inprod)

    logger.debug(self.in_products_dict[ean])

    return {CustCost.K_NAME: new_inprod.name, CustCost.K_QTY: new_inprod.bought_qty}

  """
  To update the sold_products
  """
  def _add_sold_prods_to_dict(self, sold_objs_list) :
    logger.debug(sold_objs_list)

    if ( self.out_products_dict is None) :
      self.out_products_dict = dict()

    the_objs_list = sold_objs_list.copy()
    init_len = len(sold_objs_list)
    saved_sold_prods_list = list()

    while len(the_objs_list) > 0  :
      sold_obj = the_objs_list[-1]
      the_objs_list.remove(the_objs_list[-1])

      ean = sold_obj[CustCost.K_EAN]
      qty = sold_obj[CustCost.K_QTY]

      for in_prod in self.in_products_dict[ean] :

        if (not in_prod.is_sold_out) :
          res_qty, price, acq_price = in_prod.is_a_unique_sale(qty)
          """
          split the sell-in trought more product stock
          """
          if (res_qty > 0) :
            qty -= res_qty
            sold_obj[CustCost.K_QTY] = res_qty
            the_objs_list.append(sold_obj)
          new_sold_prod = SoldProduct(sold_obj[CustCost.K_NAME], ean, qty, price, acq_price, date.today() )
          new_sold_prod.tot = qty * price

          sold_ean_list = list( self.out_products_dict.keys() )
          if ( ean in sold_ean_list) :
            the_out_list = self.out_products_dict[ean]
            the_out_list = the_out_list.append(new_sold_prod)
          else :
            self.out_products_dict[ean] = [new_sold_prod]

          saved_sold_prods_list.append(new_sold_prod)
          break

    logger.debug(self.out_products_dict)
    return saved_sold_prods_list

  def _calc_revenues(self) :

    if (self.out_products_dict is None) or len( list( self.out_products_dict.keys() ) ) == 0  :
      raise ValueError(GuiMessage.WARN_REV_NO_DATA)

    revenues = {CustCost.K_REV_TOT : 0., CustCost.K_SALES_TOT : 0.}

    revenues_l = []
    revenues_tot = 0
    sales_tot    = 0

    for key in self.out_products_dict :
      sold_prods_l = self.out_products_dict[key]

      qty            = 0
      sellin_pr_rev  = 0
      sales_pr_tot   = 0
      acq_price_mean = 0
      sellin_price_m = 0

      for sold_pr in sold_prods_l :
        qty            += sold_pr.sold_qty
        sellin_pr_rev  += ((sold_pr.sold_price - sold_pr.acq_price) * float(sold_pr.sold_qty))
        sales_pr_tot   += sold_pr.tot
        acq_price_mean += sold_pr.acq_price
        sellin_price_m += sold_pr.sold_price

      acq_price_mean = acq_price_mean / float(qty)
      sellin_price_m = sellin_price_m / float(qty)

      revenues_prod_result = {
        CustCost.K_NAME: sold_prods_l[0].name,
        CustCost.K_QTY_TOT : qty,
        CustCost.K_REV_TOT: sellin_pr_rev,
        CustCost.K_SALES_TOT : sales_pr_tot,
        CustCost.K_ACQ_PRICE_MEAN : acq_price_mean,
        CustCost.K_SELLIN_PRICE_MEAN : sellin_price_m
        }
      if (revenues_l is not None) :
        revenues_l = revenues_l.append(revenues_prod_result)
      else :
        revenues_l = [revenues_prod_result]

      revenues_tot += sellin_pr_rev
      sales_tot    += sales_pr_tot


    revenues = {CustCost.K_REV_TOT : revenues_tot, CustCost.K_SALES_TOT : sales_tot}

    return revenues_l, revenues


In [ ]:
logger.setLevel(logging.INFO)
GUIController()

INFO:APP_STORE:aiuto:	manuale utente



BENVENUT* da 'BioMarket s.a.s.' nel negozio TAN_STORE di Via Tan 6 

I comandi abilitati sono i seguenti: 
- aiuto:  manuale utente
- aggiungi:  aggiungi un prodotto al magazzino
- elenca:  elenca i prodotto in magazzino
- vendita:  registra una vendita effettuata
- profitti:  mostra i profitti totali
- chiudi:  esci dal programma

Inserisci un comando: profitti


INFO:APP_STORE:profitti:	mostra i profitti totali



Non sono presenti dati di vendita per calcolare profitti.

Inserisci un comando: vendita


INFO:APP_STORE:vendita:	registra una vendita effettuata


...

INFO:APP_STORE:elenca:	elenca i prodotto in magazzino



I Prodotti presenti sono: 
PRODOTTO                           	QUANTITà  	PREZZO
Latte di Soia                      	        15	2.10
Latte di Mandorle                  	        30	2.30
Orzo perlato                       	        15	1.75
Gallette di Riso                   	        35	1.99
Riso Venere                        	        10	2.25

Inserisci un comando: chiudi


INFO:APP_STORE:chiudi:	esci dal programma


Vuoi realmente uscire?s
Bye bye


SystemExit: 